# Sending Insights to Remote Server

This example walks you through the API for sending insights to remote metrics servers.

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
import os

LOCAL_FOLDER = f"{os.getcwd()}/."
TEMPO_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))

### Custom inference logic 

Our custom model will be very simple to focus the logic on the insights functionality.


In [ ]:
# %load src/tempo.py
import os
import json
import numpy as np
from tempo.serve.utils import pipeline, predictmethod
from tempo.serve.metadata import RuntimeOptions, InsightRequestModes
from tempo.utils import logger
from tempo.insights.context import insights

@pipeline(
    name='insights-pipeline',
    uri="s3://tempo/divorce",
    local_folder=LOCAL_FOLDER,
)
class Pipeline:
    
    @predictmethod
    def predict(self, data: np.ndarray, parameters: dict) -> np.ndarray:
        if parameters.get("log"):
            insights.log_request()
            insights.log_response()
            insights.log(parameters)
        return data


In [ ]:
pipeline = Pipeline()

### Explicitly Log Insights 
We explicitly request to log by passing the parameters

In [ ]:
params = { "log": "value" }
data = np.array([63])
pred = pipeline(data, params)
print(pred)

### Don't log insights

In [ ]:
params = { }
data = np.array([63])
pred = pipeline(data, params)
print(pred)

### Deploy the  Model to Docker

Finally, we'll be able to deploy our model using Tempo against one of the available runtimes (i.e. Kubernetes, Docker or Seldon Deploy).

We'll deploy first to Docker to test.

In [ ]:
%%writetemplate $LOCAL_FOLDER/conda.yaml
name: tempo-insights
channels:
  - defaults
dependencies:
  - pip=21.0.1
  - python=3.7.9
  - pip:
    - mlops-tempo @ file://{TEMPO_DIR}
    - mlserver==0.3.1.dev7

In [ ]:
from tempo.serve.loader import save
save(pipeline, save_env=True)

In [ ]:
from tempo.seldon import SeldonDockerRuntime

docker_runtime = SeldonDockerRuntime()
docker_runtime.deploy(pipeline)
docker_runtime.wait_ready(pipeline)

We can now test our model deployed in Docker as:

## Log insights

In [ ]:
params = { "log": "value" }
data = np.array([63])
pipeline.remote(data=data, parameters=params)

## Don't log

In [ ]:
params = { }
data = np.array([63])
pipeline.remote(data=data, parameters=params)

In [ ]:
docker_runtime.undeploy(pipeline)

## Production Option 1 (Deploy to Kubernetes with Tempo)

 * Here we illustrate how to run the final models in "production" on Kubernetes by using Tempo to deploy
 
### Prerequisites
 
 Create a Kind Kubernetes cluster with Minio and Seldon Core installed using Ansible from the Tempo project Ansible playbook.
 
 ```
 ansible-playbook ansible/playbooks/default.yaml
 ```

In [ ]:
!kubectl apply -f k8s/rbac -n production

In [ ]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [ ]:
from tempo.serve.loader import upload
upload(pipeline)

In [ ]:
from tempo.serve.metadata import RuntimeOptions, KubernetesOptions
runtime_options = RuntimeOptions(
        k8s_options=KubernetesOptions(
            namespace="production",
            authSecretName="minio-secret"
        )
    )

In [ ]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
k8s_runtime = SeldonKubernetesRuntime(runtime_options)
k8s_runtime.deploy(pipeline)
k8s_runtime.wait_ready(pipeline)

## Log insights

In [ ]:
params = { "log": "value" }
data = np.array([63])
pipeline.remote(data=data, parameters=params)

## Don't log

In [ ]:
params = {  }
data = np.array([63])
pipeline.remote(data=data, parameters=params)